In [ ]:
import json
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

# OpenAI 임베딩 설정
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 스토어 저장 경로
VECTOR_STORE_DIR = "vector_store"


# JSON 데이터를 벡터 스토어에 저장하는 함수
def json_to_vector_store(json_filename, store_name):
    with open(json_filename, "r", encoding="utf-8") as f:
        total = json.load(f)

    documents = []
    for idx in range(len(total)):
        content = total[idx]
        merged_text = f"type: {content['type']}, platform: {content['platform']}, title: {content['title']}, status: {content['status']}, genre: {content['genre']}, views: {content['views']}, description: {content['description']}, keyword: {content['keywords']}"
        if isinstance(content["keywords"], str):
            keywords = content["keywords"]
        else:
            keywords = ", ".join(content["keywords"])
        documents.append(
            Document(
                page_content=merged_text,
                metadata={
                    "type": content["type"],
                    "platform": content["platform"],
                    "title": content["title"],
                    "status": content["status"],
                    "genre": content["genre"],
                    "views": content["views"],
                    "description": content["description"],
                    "keywords": keywords,
                },
            )
        )

        from pprint import pprint

        pprint(documents)

    # 벡터 저장소 생성 및 저장
    store_path = os.path.join(VECTOR_STORE_DIR, store_name)
    vector_store = Chroma.from_documents(
        documents=documents, embedding=embedding_model, persist_directory=store_path
    )
    vector_store.persist()


# 각 JSON 파일을 벡터로 변환하여 저장
json_to_vector_store(
    "C:/Project/Final_test/data/Raw_DB/total_webtoon.json", "webtoon_vector_store"
)
json_to_vector_store(
    "C:/Project/Final_test/data/Raw_DB/total_webnovel.json", "webnovel_vector_store"
)
json_to_vector_store(
    "C:/Project/Final_test/data/Raw_DB/total.json", "total_vector_store"
)

print("JSON 파일들을 벡터로 변환하여 저장 완료.")

In [ ]:
# 벡터 스토어 로드
webtoon_vector_store = Chroma(
    persist_directory="vector_store/webtoon_vector_store",
    embedding_function=embedding_model,
)
webnovel_vector_store = Chroma(
    persist_directory="vector_store/webnovel_vector_store",
    embedding_function=embedding_model,
)
total_vector_store = Chroma(
    persist_directory="vector_store/total_vector_store",
    embedding_function=embedding_model,
)


def check_vector_store(vector_store, store_name, query="자고 일어났더니"):
    print(f"\n=== {store_name} 벡터 스토어 내용 확인 ===")
    try:
        results = vector_store.similarity_search(query)
        print(results[0] if results else "검색 결과 없음")
    except Exception as e:
        print(f"검색 오류: {e}")


# 벡터 스토어 내용 확인
check_vector_store(webtoon_vector_store, "웹툰")
check_vector_store(webnovel_vector_store, "웹소설")
check_vector_store(total_vector_store, "통합 데이터")

print("\n 벡터 스토어 검색 완료.")